### 유튜브 api

In [1]:
# !pip install scrapetube
# !pip install google-api-python-client

In [3]:
import pandas as pd
import datetime
import scrapetube
from googleapiclient.discovery import build

In [ ]:
class YoutubeAPI:
    def __init__ (self):
        self.api_key = 'api_key'
        self.youtube = build('youtube', 'v3', developerKey=api_key)
        channel_id = 'UChlv4GSd7OQl3js-jkLOnFA' # 삼프로TV
        
    def get_video_ids(self):
        videos = scrapetube.get_channel(self.channel_id)
        
        video_ids = []
        for video in videos:
            video_ids.append(video['videoId'])
        
        self.get_video_infos(video_ids)
    
    def get_video_infos(self, video_ids):
        video_infos = []
        
        for i in range(50):
            start = i * 50
            end = (i + 1) * 50
            
            video_request = self.youtube.videos().list(
                part='snippet',
                id=','.join(video_ids[start:end]))
            
            video_response = video_request.execute()
            
            for item in video_response['items']:
                title = item['snippet']['title']
                if ('글로벌 이슈체크' in title) or ('글로벌 마켓브리핑' in title) or ('직장인 vlog' in title):
                    continue
                video_infos.append([item['snippet']['title'], item['snippet']['publishAt'], item['id']])
    
            df_ids = pd.DataFrame(video_infos, columns=['title', 'video_date', 'id'])
            
            self.get_comments(df_ids)
            
    def get_comments(self, df_ids):
        comments = []
        
        for video_id in df_ids['id']:
            api_obj = build('youtube', 'v3', developerKey=self.api_key)
            response = api_obj.commentThreads().list(part='snippet', videoId=video_id, maxResults=100).execute()
            
            while response:
                for item in response['items']:
                    comment = item['snippet']['topLevelComment']['snippet']
                    comments.append([video_id, comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
            
            if 'nextPageToken' in response:
                response = api_obj.commentThreads().list(part='snippet', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
            else:
                break
                
        df_comments = pd.DataFrame(comments, columns=['id', 'comment', 'author', 'comment_date', 'num_likes'])
        
        df = pd.merge(df_comments, df_ids, on='id', how='outer')
        
        self.update_db(df)
        
    def update_db(self, df):
        pass
    
    def execute_daily(self):
        pass